# Gait Data Analysis Pipeline  
Load gait‐parameter CSVs and reference data, merge & compare them, compute RMSE and save the data as a excel file.


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets, Output
from IPython.display import display

# --- Settings ---
trials = [f"TEST-{chr(c)}" for c in range(65, 77)]  # TEST-A to TEST-L
base_path = "/path/to/trials"  # root folder
output_file = os.path.join(base_path, "gait_analysis_results.xlsx")

## 1. Load Algorithm CSVs

In [ ]:
# Step 1: Read per-trial CSVs into algo_data dict
algo_data = {}
for trial in trials:
    csv_path = os.path.join(base_path, trial, "Results", "Figures", f"{trial}_parameters.csv")
    try:
        df = pd.read_csv(csv_path, header=[0,1])
        # clean, re-index...
        algo_data[trial] = df
    except FileNotFoundError:
        print(f"CSV missing for {trial}")


## 2. Load Reference (Awinda) Excel Summaries

In [ ]:
# Step 2: Read per-trial Excel into ref_data dict
ref_data = {}
for trial in trials:
    xls_path = os.path.join(base_path, trial, "Awinda Summarized.xlsx")
    try:
        df = pd.read_excel(xls_path, header=[1,2,3], index_col=0)
        # clean, stack, filter mean...
        ref_data[trial] = df
    except FileNotFoundError:
        print(f"Excel missing for {trial}")


## 3. Merge Algorithm Data and Reference Data


In [ ]:
# Step 3: For each trial, merge algo_data[trial] with ref_data[trial]
merged_list = []
for trial in trials:
    if trial in algo_data and trial in ref_data:
        df_a = algo_data[trial]  # reset_index, filter bilateral
        df_r = ref_data[trial]
        df_m = pd.merge(df_r, df_a, on=['Trial','Task','Foot'], suffixes=('_ref','_algo'))
        merged_list.append((trial, df_a, df_r, df_m))


## 4. Compute RMSE Summary

In [ ]:
# Step 4: Concatenate all merges and compute task-level RMSE
all_merged = pd.concat([m for _,_,_,m in merged_list], ignore_index=True)
df_bi = all_merged[all_merged['Foot']=='bilateral']
gait_params = ['cadence','stance_time','swing_time', 'stride_length', 'stride_width']
rmse_df = pd.DataFrame(index=df_bi['Task'].unique(), columns=gait_params)

for p in gait_params:
    x = df_bi[f"{p}_mean_ref"]
    y = df_bi[f"{p}_mean_algo"]
    rmse_df[p] = np.sqrt(((x-y)**2).groupby(df_bi['Task']).mean())
rmse_df['n_trials'] = df_bi.groupby('Task').size()


## 5. Export to Excel Workbook

In [ ]:
# Step 5: Save each sheet into a single .xlsx
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for trial, df_a, df_r, df_m in merged_list:
        df_a.to_excel(writer, sheet_name=f"{trial}_algo")
        df_r.to_excel(writer, sheet_name=f"{trial}_ref", index=False)
        df_m.to_excel(writer, sheet_name=f"{trial}_merged", index=False)
    all_merged.to_excel(writer, sheet_name="All_Merged", index=False)
    rmse_df.to_excel(writer, sheet_name="RMSE_Summary")
print("Workbook saved:", output_file)
